In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

# OPEN AND LOGIN TWITTER/ X

In [2]:
driver = webdriver.Edge()

WebDriverException: Message: 'MicrosoftWebDriver.exe' executable needs to be in PATH. Please download from http://go.microsoft.com/fwlink/?LinkId=619687


In [23]:
driver.get('https://x.com/login')

In [24]:
username = input()


In [25]:
import getpass

passTxt = str(getpass.getpass())

In [26]:
input_username = driver.find_element(By.TAG_NAME, "input")
input_username.clear()
input_username.send_keys(username)

In [27]:
signin = driver.find_elements(By.XPATH, "//div[@role = 'button']")

In [28]:
signin[2].click()

In [29]:
password = driver.find_elements(By.TAG_NAME, "input")

In [30]:
password[1].clear()
password[1].send_keys(passTxt)

In [31]:
submit = driver.find_elements(By.XPATH, "//div[@role = 'button']")

In [32]:
submit[2].click()

# CRAWLING COMMUNITY MEMBERS

In [30]:
import time
# get community member
driver.get('https://x.com/i/communities/1779743686756299180/members')
last_height = driver.execute_script("return document.body.scrollHeight")
i = 0
members_dict = {"name":[], "username":[]}

while (True):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    members = driver.find_elements(By.XPATH, "//div[@data-testid = 'cellInnerDiv']")
    print(len(members))
    for m in members:
        if("@" in m.text):
            if (m.text.splitlines()[0] not in members_dict["name"]):
                members_dict["name"].append(m.text.splitlines()[0])
                if(len(m.text.splitlines()) == 4):
                    members_dict["username"].append(m.text.splitlines()[2])
                else:
                    members_dict["username"].append(m.text.splitlines()[1])
                print(m.text.splitlines())
                print("--------")
            else:
                 print(f"{m.text.splitlines()[0]} already exist") 
        # Calculate new scroll height and compare with last scroll height.
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
            break

    last_height = new_height

print("Finished Crawling Members List")

21
['Binus Fess', 'Admin', '@binusmenfess', 'Ikuti']
--------
['r', '@piledriverwultz', 'Ikuti']
--------
['anakpantai', '@morningandcat', 'Ikuti']
--------
['aten', 'Mod', '@atenkatroz', 'Ikuti']
--------
['juannn', '@juanwmv', 'Ikuti']
--------
['aiko', '@aj05j8', 'Ikuti']
--------
['aul | Freebies TDS3', '@jungauli', 'Ikuti']
--------
['b', '@iambirubi', 'Ikuti']
--------
['Archie', '@ArchieRawr', 'Ikuti']
--------
['vieee', 'Mod', '@strawb3ery', 'Ikuti']
--------
['kevinsfඞ', '@krvinSFF', 'Ikuti']
--------
['clara de sforza', '@clarazula', 'Ikuti']
--------
['(willsens)', '@Willsens2', 'Ikuti']
--------
['yasun', '@mocsunJA', 'Ikuti']
--------
['saras', '@dapsrst', 'Ikuti']
--------
['rere 𖤐⭒๋࣭ ⭑', '@tiaphariteler', 'Ikuti']
--------
['Vyn', '@munyamunya88', 'Ikuti']
--------
['Rene', '@iessthers', 'Ikuti']
--------
['a', '@XXenatha', 'Ikuti']
--------
['kani', '@lowdydoo', 'Ikuti']
--------
41
Binus Fess already exist
r already exist
anakpantai already exist
aten already exist
jua

In [31]:
df = pd.DataFrame(members_dict)

In [32]:
df

,name,username
0,Binus Fess,@binusmenfess
1,r,@piledriverwultz
2,anakpantai,@morningandcat
3,aten,@atenkatroz
4,juannn,@juanwmv
...,...,...
190,grace,@antifragrace
191,fad,@jenamourrr
192,@aventurine_wz,Ikuti
193,saa pi mo mo,@snowlyie


In [33]:
df.to_csv("binus_fess_members.csv",index=False)

# Crawling Account Followers

In [33]:
def crawl_followers(user_account, dicts, endpoints):
    driver.get(f'https://x.com/{user_account}/{endpoints}')
    last_height = driver.execute_script("return document.body.scrollHeight")
    while (True):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        followers = driver.find_elements(By.XPATH, "//div[@data-testid = 'cellInnerDiv']")
        print(len(followers))
        for m in followers:
            if("@" in m.text):
                if (m.text.splitlines()[0] not in dicts["name"]):
                    dicts["name"].append(m.text.splitlines()[0])
                    dicts["username"].append(m.text.splitlines()[1])
                    print(m.text.splitlines())
                    print("--------")
                else:
                    print(f"{m.text.splitlines()[0]} already exist") 
            # Calculate new scroll height and compare with last scroll height.
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break

        last_height = new_height

    print("Finished Crawling Members List")
    return dicts

In [40]:
import time
# get community member
user_accounts = ["binusmenfess", "undipmenfess","itbfess","UPIfess","UGM_FESS"]
followers_dict = {"name":[], "username":[]}
for u in user_accounts:
    print(f"crawling:  {u}")
    followers_dict = crawl_followers(u,followers_dict,"followers")
    followers_dict = crawl_followers(u,followers_dict,"verified_followers")
    followers_dict = crawl_followers(u,followers_dict,"followers_you_follow")
    df_followers = pd.DataFrame(followers_dict)
    print(df_followers.head())
    df_followers.to_csv(f"{u}_followers.csv",index=False)

crawling:  binusmenfess
16
['mohamad yunus', '@sunuymuhammad', 'Ikuti']
--------
['𝐑𝐡𝐞', '@rhestudy', 'Ikuti']
--------
['jihanasyifa', '@jihanasyifaaa', 'Ikuti']
--------
['HoU lelah hidup', '@bengbenggorenk', 'Ikuti', 'ternak naga sama penguin']
--------
['liv ☽︎', '@vanillalav', 'Ikuti', '— Information Systems Student, B26']
--------
['nanda', '@Nanda160304', 'Ikuti', 'Hddhhjxjxhsjsjs']
--------
['Yera', '@yerz__', 'Ikuti', 'some wild sunibian u found on the street playing Honkai Star Rail']
--------
['aya', '@owhkotsu', 'Ikuti', 'kunang-kunang kuliah nangis kuliah nangis']
--------
['yy 𖦹 ˖°⋆ ଓ', '@BUSlNESSLAW', 'Ikuti', 'HORRASSSS MEDAANN']
--------
['mouw', '@meii89046', 'Ikuti']
--------
["CAMABA FISIP'24", '@cessatea', 'Ikuti', 'CAMABA UNPAD 2024 aamiinn']
--------
['( ´◡‿ゝ◡`)', '@keonariel', 'Ikuti']
--------
['pikacuhi', '@pikaacuhii', 'Ikuti']
--------
['Seolalie', '@seolaliee', 'Ikuti', 'BN27 DKV ANIMATION']
--------
['@Fluorine_heit', 'Ikuti', 'Caffeine powered nerd']
----

In [38]:
len(followers_dict["username"])

39

5
['𝐌.', '@uhutao', 'Ikuti', 'Ⓘ ᴛʜɪꜱ ᴜꜱᴇʀ ɪꜱ ᴄᴜʀʀᴇɴᴛʟʏ ᴜɴᴅᴇʀ ᴍᴀɪɴᴛᴇɴᴀɴᴄᴇ ꜰᴏʀ ꜰᴜʀᴛʜᴇʀ ᴘʀᴏɢʀᴇꜱꜱ Ⓘ']
--------
['FIO', '@0208JPG', 'Ikuti']
--------
['エノン (✿^‿^)', '@enonahana', 'Ikuti', 'book , nanami ♡, food // call me enon ♡']
--------
['Victor “Viper” Chandra', '@mzmznasipadang', 'Ikuti', 'Tukang Foto Pesawat dan Mahasiswa CS dan SI']
--------
5
𝐌. already exist
FIO already exist
エノン (✿^‿^) already exist
Victor “Viper” Chandra already exist
Finished Crawling Members List
2
['BINUS Kemanggisan', '@BINUS_KMG', 'Mengikuti', 'Official account of Binus Kemanggisan. Binus Kemanggisan consists of 4 buildings, Anggrek Campus, Syahdan Campus, Kijang Campus, and Binus Square.']
--------
2
BINUS Kemanggisan already exist
Finished Crawling Members List


# CRAWLING TWEET

In [13]:
# driver.get('https://x.com/sweetersner')

def is_protected():
    import time
    time.sleep(2)
    divs = driver.find_elements(By.XPATH,"//div[@dir='ltr']")
    if len(divs) > 0:
        divs_all = [div.text if str(div.text) else "skip" for div in divs]
        if("Postingan ini dilindungi" in divs_all):
            print("protected")
            return True
    return False

In [16]:
"0 postingan" == "0 postingan"

True

In [17]:
def checkPostExist():
    infopost = driver.find_elements(By.XPATH, "//div[@dir = 'ltr']")
    # print(len(infopost))
    for i in infopost:
        if(len(i.text.splitlines()) > 0):
            if "0 postingan" == i.text.splitlines()[0]:
                print(i.text)
                return False
    return True
    

In [18]:
def import_data():
    import json
    import os
    json_file_path = "craw_data.json"
    datas = {}
    if os.path.exists(json_file_path):
        with open(json_file_path, 'r') as json_file:
            # Step 5: Read the JSON file and convert it to a dictionary
            datas = json.load(json_file)
        print(datas.keys())
    else:
        print("File does not exist. new dictionary created")
        datas = {}
    return datas

In [19]:
def save_dict(datas):
    import json

    # Convert the dictionary to a JSON string
    json_data = json.dumps(datas)
    print("checking keys of dictionary: ")
    print(datas.keys())
    # Save the JSON to a file
    if(len(datas.keys()) > 0):
        with open("craw_data.json", "w") as file:
            file.write(json_data)
    else:
        print("cannot save file, no data in dictionary")

In [20]:
def crawl(datas,x_account, limit, with_replies):
    import time
    if not with_replies:
        driver.get('https://x.com/'+x_account)
    else:
        driver.get('https://x.com/'+x_account+"/with_replies")
    protected = False
    try:
        protected = (is_protected() or (not checkPostExist()))
    except:
        import sys
        print("Error")
        save_dict(datas)
        sys.exit()
    if(protected):
        print("protected or has no post")
        return False, True
    # Get scroll height.
    last_height = driver.execute_script("return document.body.scrollHeight")
    i = 0
    while (i < limit):

        # Scroll down to the bottom.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load the page.
        time.sleep(5)
        tweet = driver.find_elements(By.XPATH, "//div[@data-testid = 'tweetText']")
        print(len(tweet))
        if(len(tweet) == 0):
            print("============== no tweet found, return to account loop =============")
            return False, False
        
        for t in tweet:
            if i > limit:
                break
            print(t.text)
            i+=1
            if not with_replies:
                if t.text not in datas[x_account]["post"]:
                    datas[x_account]["post"].append(t.text.lower())
            else:
                if t.text not in datas[x_account]["replies"]:
                    datas[x_account]["replies"].append(t.text.lower())
            print("-----------------------------")
        # Calculate new scroll height and compare with last scroll height.
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:

            break

        last_height = new_height
    return True, False
        

In [21]:
datas = import_data()
skip = False
crawl_count = user_count = 0
# users = pd.read_csv("users.csv")["username"].tolist()
users = pd.read_csv("binus_fess_members.csv")["username"].tolist()

dict_keys(['@yeoracula', '@ai_istired', '@pacarddoy', '@tatanies', '@krvinSFF', '@milkhoneysweett', '@forcevield', '@oresanna', '@jsicasthanie', '@clarazula', '@xorain_fluffy', '@vandh_', '@wittyhog', '@dreamovermorals', '@seanne_7', '@nanakimbapppp', '@sebewntin', '@parkeujust', '@nagetdinogeprek', '@hanksmumma', '@cexsaaa', '@Chapibaruyaa', '@bukangirlboss', '@piledriverwultz', '@morningandcat', '@juanwmv', '@aj05j8', '@jungauli', '@ArchieRawr', '@Willsens2', '@mocsunJA', '@dapsrst', '@tiaphariteler', '@munyamunya88', '@iessthers', '@lowdydoo', '@XXenatha', '@lanamaulanna', '@akuanakhooman', '@martabakejoe', '@anakomaliong', '@enatdeh', '@capekbgt124', '@gensetterz', '@gabbinhere', '@seericheese', '@geez_ann_', '@polayoid', '@DuncanDonut_', '@hewhoflauntsred', '@aikiropass', '@nishimiow', '@ROisinitial', '@mumbocutie', '@peterrtingle', '@Wh1zKt', '@aizzzzh', '@BaronBram', '@sasalelah', '@tunaable', '@electnibro', '@DlMABUK', '@lesyeuxdeasaa', '@GtMiyye', '@kaiinotherern', '@baratgasw

In [27]:
for x_account in users:
    import time
    limit = 200
    print(f"crawling: {x_account}")
    if x_account not in datas.keys() or ((len(datas[x_account]["post"]) == 0) and (len(datas[x_account]["replies"]) == 0)):
        datas[x_account] = {}
        datas[x_account]["post"] = []
        datas[x_account]["replies"] = []
        protected = False
        print("---------- crawling post -----------------")
        cond1, protected = crawl(datas, x_account, limit, False)
        if protected:
            print(f"skipping user: {x_account}")
            datas.pop(x_account)
            skip = True
            continue
        print("---------- crawling replies -----------------")
        cond2, protected = crawl(datas, x_account, limit, True)
        if protected:
            print(f"skipping user: {x_account}")
            datas.pop(x_account)
            skip = True
            continue
        if(not cond1):
            print("!! ----- 6 minute sleep to retry post twitter view count ----- !!")
            time.sleep(360)
            cond1 = crawl(datas, x_account, limit, False)
            skip = False
        if(not cond2):
            print("!! ----- 6 minute sleep to retry replies twitter view count ----- !!")
            time.sleep(360)
            cond2 = crawl(datas, x_account, limit, True)
            skip = False
        cond = cond1 and cond2
        if cond:
            time.sleep(10)
            skip = False
            crawl_count+=1
        else:
            print("saving data because of twitter error")
            datas.pop(x_account)
            save_dict(datas)
            break
    else:
        print(f"skipping user: {x_account}")
        skip = True
        continue
    user_count+=1
print("saving all data")
save_dict(datas)

crawling: @binusmenfess
skipping user: @binusmenfess
crawling: @piledriverwultz
skipping user: @piledriverwultz
crawling: @morningandcat
skipping user: @morningandcat
crawling: @atenkatroz
skipping user: @atenkatroz
crawling: @juanwmv
skipping user: @juanwmv
crawling: @aj05j8
skipping user: @aj05j8
crawling: @jungauli
skipping user: @jungauli
crawling: @iambirubi
skipping user: @iambirubi
crawling: @ArchieRawr
skipping user: @ArchieRawr
crawling: @strawb3ery
skipping user: @strawb3ery
crawling: @krvinSFF
skipping user: @krvinSFF
crawling: @clarazula
skipping user: @clarazula
crawling: @Willsens2
skipping user: @Willsens2
crawling: @mocsunJA
skipping user: @mocsunJA
crawling: @dapsrst
skipping user: @dapsrst
crawling: @tiaphariteler
skipping user: @tiaphariteler
crawling: @munyamunya88
skipping user: @munyamunya88
crawling: @iessthers
skipping user: @iessthers
crawling: @XXenatha
skipping user: @XXenatha
crawling: @lowdydoo
skipping user: @lowdydoo
crawling: @binusmenfess
skipping user:

AssertionError: 

In [ ]:
import pandas as pd
df = pd.DataFrame(datas)
df.to_csv("datas.csv")